In [1]:
from openai import OpenAI
from datetime import datetime, timedelta
import random
import pandas as pd
import os
from dotenv import load_dotenv

In [9]:
from tqdm.notebook import tqdm


In [2]:
# Betöltjük az OpenAI API kulcsot
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Tech brandek és termékek
brands = ['Samsung', 'Apple', 'Sony', 'LG', 'Dell', 'HP', 'Lenovo', 'Asus']
products = ['telefon', 'laptop', 'tévé', 'monitor', 'tablet', 'okosóra']

In [3]:
# Üzenetgeneráló prompt template
def generate_openai_message(role, brand, product, context=None):
    if role == "user":
        prompt = f"""Írj egy vásárlói panasz üzenetet, amely tartalmazza a(z) {brand} márkájú {product} terméket. 
A stílus legyen természetes, mintha egy átlagos vásárló írná meg. Ügyelj rá, hogy ne sablonos legyen. 
{"Az előző üzenet: " + context if context else ""}"""
    else:
        prompt = f"""Írj egy udvarias, segítőkész ügyfélszolgálati választ egy olyan panaszra, amely a(z) {brand} {product} termékkel kapcsolatos. 
A válasz legyen természetes hangvételű, mintha egy emberi ügyfélszolgálatos írta volna. 
{"Az előző ügyfélüzenet: " + context if context else ""}"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Te egy ügyfélszolgálati beszélgetés szimulációját segítő nyelvi modell vagy."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [4]:
# Egy beszélgetés generálása
def generate_openai_chat_conversation():
    user_id = f"user_{random.randint(1, 20)}"
    agent_id = f"agent_{random.randint(1, 5)}"
    base_time = datetime.now() - timedelta(days=random.randint(0, 30), hours=random.randint(0, 23))

    brand = random.choice(brands)
    product = random.choice(products)

    messages = []
    context = ""
    time_step = 0

    # Több üzenetből álló beszélgetés generálása
    for _ in range(random.randint(2, 4)):
        user_msg = generate_openai_message("user", brand, product, context)
        user_time = base_time + timedelta(minutes=time_step)
        messages.append({
            "timestamp": user_time,
            "sender_type": "user",
            "sender_id": user_id,
            "message": user_msg,
            "reply_to": None
        })
        context = user_msg
        time_step += 5

        agent_msg = generate_openai_message("agent", brand, product, context)
        agent_time = base_time + timedelta(minutes=time_step)
        messages.append({
            "timestamp": agent_time,
            "sender_type": "agent",
            "sender_id": agent_id,
            "message": agent_msg,
            "reply_to": user_id
        })
        context = agent_msg
        time_step += 5

    return messages

In [ ]:
# Több beszélgetés szimulálása
all_messages = []
for _ in tqdm(range(10), desc="Beszélgetések generálása"):
    all_messages.extend(generate_openai_chat_conversation())

Beszélgetések generálása:   0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
df_openai_chat = pd.DataFrame(all_messages)

In [8]:
df_openai_chat

,timestamp,sender_type,sender_id,message,reply_to
0,2025-05-02 01:07:12.142577,user,user_3,"Tisztelt Ügyfélszolgálat!\n\nRemélem, ez a lev...",None
1,2025-05-02 01:12:12.142577,agent,agent_5,"Tisztelt [Az Ön Neve]!\n\nKöszönjük, hogy megk...",user_3
2,2025-05-02 01:17:12.142577,user,user_3,"Kedves [Ügyfélszolgálatos Neve]!\n\nRemélem, j...",None
3,2025-05-02 01:22:12.142577,agent,agent_5,"Kedves [Az Ön Neve]!\n\nKöszönöm, hogy újra me...",user_3


In [ ]:

import random

# Véletlenszerű nevek listája
first_names = ["János", "Katalin", "Bence", "Eszter", "Zoltán", "Anna", "Gábor", "Dóra", "Miklós", "Erika"]
last_names = ["Kovács", "Szabó", "Tóth", "Varga", "Kiss", "Molnár", "Nagy", "Balogh", "Farkas", "Papp"]

# Egyedi user_id-hez hozzárendelt nevek
user_names = {}

def get_user_name(user_id):
    if user_id not in user_names:
        full_name = f"{random.choice(last_names)} {random.choice(first_names)}"
        user_names[user_id] = full_name
    return user_names[user_id]


In [ ]:

def generate_openai_message(client, role, brand, product, user_id=None, context=None):
    name = get_user_name(user_id) if user_id else "Vásárló"
    if role == "user":
        prompt = f"""Írj egy vásárlói panasz üzenetet, amely tartalmazza a(z) {brand} márkájú {product} terméket. 
A stílus legyen természetes, mintha egy átlagos vásárló írná meg. Ügyelj rá, hogy ne sablonos legyen. 
{"Az előző üzenet: " + context if context else ""}"""
    else:
        prompt = f"""Írj egy udvarias, segítőkész ügyfélszolgálati választ egy olyan panaszra, amely a(z) {brand} {product} termékkel kapcsolatos. 
A válasz szövegében szólítsd meg név szerint a vásárlót ezzel a formulával: 'Tisztelt {name}!'. 
A válasz legyen természetes hangvételű, mintha egy emberi ügyfélszolgálatos írta volna. 
{"Az előző ügyfélüzenet: " + context if context else ""}"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Te egy ügyfélszolgálati beszélgetés szimulációját segítő nyelvi modell vagy."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()
